# Cryptocurrency Clusters Machine Learning

## Data Preparation

In [ ]:
# Dependencies


In [ ]:
# Read crypto_data.csv into Pandas
# The dataset was obtained from: https://min-api.cryptocompare.com/data/all/coinlist


In [ ]:
# Discard all cryptocurrencies that are not being traded and drop the IsTrading column from the dataframe


In [ ]:
# Remove all rows that have at least one null value


In [ ]:
# Filter for cryptocurrencies that have been mined (the total coins mined should be greater than zero)


In [ ]:
# Delete the CoinName from the original dataframe


In [ ]:
# Convert Algorithm and ProofType into numerical data
# Use Pandas to create dummy variables
# Examine the number of rows and columns of your dataset now. How did they change?


### Changes:

In [ ]:
# Standardize the dataset so columns that contain larger values do not unduly influence the outcome


## Dimensionality Reduction

In [ ]:
# Perform dimensionality reduction with PCA (preserve 90% of the explained variance)

# (Rather than specify the number of principal components when you instantiate the PCA model,
# it is possible to state the desired explained variance. For example, say that a dataset has 100 features.
# Using PCA(n_components=0.99) creates a model that will preserve approximately 99% of the explained variance,
# whether that means reducing the dataset to 80 principal components or 3. For this project,
# preserve 90% of the explained variance in dimensionality reduction.)

### Changes:

In [ ]:
# Further reduce the dataset dimensions with t-SNE
# Run t-SNE on the principal components: the output of the PCA transformation


In [ ]:
# Create a scatter plot of the t-SNE output (observe whether there are distinct clusters or not)


## Cluster Analysis with k-Means

In [ ]:
# Create an elbow plot to identify the best number of clusters


In [ ]:
# Use a for-loop to determine the inertia for each k between 1 through 10
# Determine, if possible, where the elbow of the plot is, and at which value of k it appears


## Recommendation